In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#################Change Data list #######################################
data=pd.read_csv('/content/drive/MyDrive/AICP 연습/1020_5y_data_FINISH.csv')
print(data.shape)
print()
data=data.dropna(axis=0)
print(data.shape)

(11132, 17)

(11132, 17)


In [ ]:
data.shape
data.reset_index(drop=True, inplace=True)
data.head()

,Pcr,Stock,Name,종가,시가,고가,저가,대용가,거래량,거래대금,PER,PSR,PBR,PCR,P/EBITDAPS,Dividend Yield,Volatility
0,1471,33780,(주)케이티앤지,96300.0,100500.0,100500.0,96300.0,79520.0,745547.0,7.296529e+10,12.47,4.05,1.86,11.03,9.03,3.82,21.57
1,64847,32640,(주)LG유플러스,16800.0,17250.0,17250.0,16500.0,13140.0,1844022.0,3.116109e+10,12.86,0.60,1.69,3.22,2.90,2.38,33.35
2,86512,69960,(주)현대백화점,91000.0,88500.0,91800.0,88500.0,68710.0,89791.0,8.145079e+09,1.02,0.15,0.06,0.70,0.57,8.47,30.84
3,88102,20000,(주)한섬,34650.0,35400.0,36350.0,34600.0,25300.0,106143.0,3.739379e+09,13.95,0.99,0.93,17.05,8.38,0.90,36.97
4,162210,51900,(주)LG생활건강,1112000.0,1048000.0,1125000.0,1047000.0,816660.0,67214.0,7.368183e+10,31.29,4.92,8.92,25.11,24.63,0.76,35.95


In [ ]:
def normalize_df(df):

    for col in list(df.columns):

        mean, std = df[col].mean(), df[col].std()

        df.loc[:, col] = (df[col] -mean) /(std + 1)

    return df

a=list(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11132 entries, 0 to 11131
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Pcr             11132 non-null  object 
 1   Stock           11132 non-null  int64  
 2   Name            11132 non-null  object 
 3   종가              11132 non-null  float64
 4   시가              11132 non-null  float64
 5   고가              11132 non-null  float64
 6   저가              11132 non-null  float64
 7   대용가             11132 non-null  float64
 8   거래량             11132 non-null  float64
 9   거래대금            11132 non-null  float64
 10  PER             11132 non-null  float64
 11  PSR             11132 non-null  float64
 12  PBR             11132 non-null  float64
 13  PCR             11132 non-null  float64
 14  P/EBITDAPS      11132 non-null  float64
 15  Dividend Yield  11132 non-null  float64
 16  Volatility      11132 non-null  float64
dtypes: float64(14), int64(1), objec

In [ ]:
df = normalize_df(data.iloc[:, 3:])
df.head()

,종가,시가,고가,저가,대용가,거래량,거래대금,PER,PSR,PBR,PCR,P/EBITDAPS,Dividend Yield,Volatility
0,-0.269104,-0.248904,-0.256406,-0.262060,-0.231692,0.002092,0.069545,-0.434879,0.266372,-0.119227,-0.076769,-0.296589,0.767661,-1.139185
1,-0.646417,-0.643420,-0.646194,-0.645382,-0.635234,0.401617,-0.143055,-0.421849,-0.364780,-0.168723,-0.111431,-0.677175,0.164256,-0.143984
2,-0.294258,-0.305771,-0.297141,-0.299527,-0.297409,-0.236412,-0.260105,-0.817455,-0.447104,-0.643311,-0.122615,-0.821835,2.716157,-0.356035
3,-0.561699,-0.557408,-0.556765,-0.558438,-0.561310,-0.230464,-0.282511,-0.385429,-0.293433,-0.390004,-0.050052,-0.336944,-0.455911,0.161841
4,4.551482,4.241237,4.540433,4.304659,4.249587,-0.244623,0.073189,0.193947,0.425532,1.936350,-0.014280,0.671951,-0.514575,0.075669


In [ ]:
df.shape

(11132, 14)

In [ ]:
df.reset_index(drop=True, inplace=True)
df=pd.concat([data.iloc[:,:3], df], axis=1)
# df.reset_index(drop=True, inplace=True)
print(df.shape)
df

(11132, 17)


,Pcr,Stock,Name,종가,시가,고가,저가,대용가,거래량,거래대금,PER,PSR,PBR,PCR,P/EBITDAPS,Dividend Yield,Volatility
0,1471,33780,(주)케이티앤지,-0.269104,-0.248904,-0.256406,-0.262060,-0.231692,0.002092,0.069545,-0.434879,0.266372,-0.119227,-0.076769,-0.296589,0.767661,-1.139185
1,64847,32640,(주)LG유플러스,-0.646417,-0.643420,-0.646194,-0.645382,-0.635234,0.401617,-0.143055,-0.421849,-0.364780,-0.168723,-0.111431,-0.677175,0.164256,-0.143984
2,86512,69960,(주)현대백화점,-0.294258,-0.305771,-0.297141,-0.299527,-0.297409,-0.236412,-0.260105,-0.817455,-0.447104,-0.643311,-0.122615,-0.821835,2.716157,-0.356035
3,88102,20000,(주)한섬,-0.561699,-0.557408,-0.556765,-0.558438,-0.561310,-0.230464,-0.282511,-0.385429,-0.293433,-0.390004,-0.050052,-0.336944,-0.455911,0.161841
4,162210,51900,(주)LG생활건강,4.551482,4.241237,4.540433,4.304659,4.249587,-0.244623,0.073189,0.193947,0.425532,1.936350,-0.014280,0.671951,-0.514575,0.075669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127,H32072,139480,(주)이마트,-0.389654,-0.386333,-0.388442,-0.387912,-0.391698,-0.244402,-0.277019,-0.789722,-0.452593,-0.605461,-0.103132,-0.706976,0.302536,-0.428689
11128,I66472,161390,한국타이어앤테크놀로지(주),-0.549834,-0.544613,-0.548337,-0.546909,-0.541188,-0.205763,-0.268497,-0.276503,-0.234891,-0.387092,-0.051206,1.208373,0.055307,-0.205656
11129,IQ7228,271560,(주)오리온,-0.174657,-0.165972,-0.168851,-0.169832,-0.149925,-0.245506,-0.263085,0.614613,0.420044,0.797921,0.026684,0.675676,-0.489433,-0.373776
11130,IV0057,280360,롯데웰푸드(주),-0.208829,-0.199619,-0.207712,-0.210662,-0.206463,-0.267089,-0.298499,0.634994,-0.412345,-0.509378,-0.061458,-0.570387,-0.003357,-1.235494


## Individual_Data

In [ ]:
len(a)

17

In [ ]:
a=[df]
a_name=['aa']

# Network

In [ ]:
if torch.cuda.is_available():
  Device=torch.device('cuda')
else:
  Device=torch.device('cpu')

device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class Factors(nn.Module):
    def __init__(self, encoding_dim):
        super(Factors, self).__init__()
        #encoder
        self.fc1 = nn.Linear(no_of_stocks, encoding_dim)

        #decoder
        self.fc2 = nn.Linear(encoding_dim, no_of_metrics)

        #encoder
        self.fc3 = nn.Linear(no_of_metrics_flat, encoding_dim)

        #decoder
        self.fc4 = nn.Linear(encoding_dim, no_of_metrics_flat)


    def forward(self, x, y):

        x = F.relu(self.fc1(x))

        x = (self.fc2(x))

        y = torch.flatten(y)

        y = F.relu(self.fc3(y))

        y = F.relu(self.fc4(y))

        y = y.view(no_of_metrics, no_of_stocks)
        x = x.view(1,no_of_metrics)

        w = torch.mm(x, y)

        return w

In [ ]:
import time
start=time.time()

In [ ]:
whole_data=pd.DataFrame()

for j, name in zip(a, a_name):
  # df_1=pd.DataFrame()
  x_input = j.종가.to_numpy()
  x = x_input.reshape(1, len(j.종가))
  print(x.shape)

  y = j.iloc[:,3:16].to_numpy()
  print(y.shape)

  print()

# initialize the NN

  no_of_stocks = len(j.종가)
  no_of_metrics = 13
  no_of_metrics_flat = no_of_stocks * no_of_metrics  #metrics.shape[1]
  encoding_dim = 32

  model = Factors(encoding_dim)
  model.to(device) ## 수정
  print(model)


  num_workers = 0
  x_batch = no_of_stocks
  y_batch = no_of_stocks
  x_loader = torch.utils.data.DataLoader(x, batch_size=x_batch, num_workers=num_workers)
  y_loader = torch.utils.data.DataLoader(y, batch_size=y_batch, num_workers=num_workers)


  # specify loss function
  criterion = nn.MSELoss()

  # specify loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

  #df=pd.DataFrame()


  for _ in range(100):
    print('This is the data: {} and time: {}. The last one is the 1000th'.format(name, _))
    df_1=pd.DataFrame()
    number = 100


    for epoch in range(1, number+1):
    # monitor training loss
      train_loss = 0.0

      for x, y in zip(x_loader, y_loader):
          x=x.to(device)
          y=y.to(device)

          optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
          outputs = model(x.float(), y.float())

        # calculate the loss
          loss = criterion(outputs, x.float())
        # backward pass: compute gradient of the loss with respect to model parameters
          loss.backward()
        # perform a single optimization step (parameter update)
          optimizer.step()
        # update running training loss
          train_loss += loss.item()

      train_loss = train_loss/len(x_loader)
      print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch,
        train_loss
        ))

    for x, y in zip(x_loader, y_loader):
        x=x.to(device)
        y=y.to(device)

        result = model(x.float(), y.float())

        unnormal_result = result * (data.종가.std() + 1) + data.종가.mean()

        print(unnormal_result.detach().cpu().numpy()[0,:5]) # 수정

        #unnormal_result = result * (data.finalp.std() + 1) + data.finalp.mean()
        #print(unnormal_result.detach().cpu().numpy()[0,:5])
##################################################
    aa=pd.DataFrame(unnormal_result.cpu().detach().numpy()) # 수정
###################################################
    result_df=aa.T
    result_df['real_price']=data['종가']
    result_df['a']=abs(result_df[0]-result_df['real_price'])
    #AAA=AAA.sort_values('a')
    result_df['Name']=data['Name']
    result_df
####################################################
#####################################################
    print()
    headresult_df.head(11132)
    head.reset_index(drop=True, inplace=True)
    print()
####################################################
    tail=result_df.tail(11132)
    tail.reset_index(drop=True, inplace=True)
#####################################################
    all_result=pd.concat([head, tail], axis=1)
    df_1=pd.concat([df_1,all_result])
    # df.reset_index(drop=True, inplace=True)

    whole_data=pd.concat([whole_data, df_1], axis=0)
    # whole_data.drop_duplicates(inplace=True)

whole_data



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch: 96 	Training Loss: 0.015473
Epoch: 97 	Training Loss: 0.015473
Epoch: 98 	Training Loss: 0.015473
Epoch: 99 	Training Loss: 0.015473
Epoch: 100 	Training Loss: 0.015473
[  96278.6     16792.688   90983.67    34638.117 1111992.1  ]


This is the data: aa and time: 52. The last one is the 1000th
Epoch: 1 	Training Loss: 0.015473
Epoch: 2 	Training Loss: 0.015473
Epoch: 3 	Training Loss: 0.015473
Epoch: 4 	Training Loss: 0.015473
Epoch: 5 	Training Loss: 0.015473
Epoch: 6 	Training Loss: 0.015473
Epoch: 7 	Training Loss: 0.015473
Epoch: 8 	Training Loss: 0.015473
Epoch: 9 	Training Loss: 0.015473
Epoch: 10 	Training Loss: 0.015473
Epoch: 11 	Training Loss: 0.015473
Epoch: 12 	Training Loss: 0.015473
Epoch: 13 	Training Loss: 0.015473
Epoch: 14 	Training Loss: 0.015473
Epoch: 15 	Training Loss: 0.015473
Epoch: 16 	Training Loss: 0.015473
Epoch: 17 	Training Loss: 0.015473
Epoch: 18 	Training Loss: 0.015473
Epoch: 19 	Training Loss: 0.015473
Epoch:

,0,real_price,a,Name,0,real_price,a,Name
0,112394.460938,96300.0,16094.460938,(주)케이티앤지,112394.460938,96300.0,16094.460938,(주)케이티앤지
1,154718.968750,16800.0,137918.968750,(주)LG유플러스,154718.968750,16800.0,137918.968750,(주)LG유플러스
2,98744.046875,91000.0,7744.046875,(주)현대백화점,98744.046875,91000.0,7744.046875,(주)현대백화점
3,151849.218750,34650.0,117199.218750,(주)한섬,151849.218750,34650.0,117199.218750,(주)한섬
4,208000.015625,1112000.0,903999.984375,(주)LG생활건강,208000.015625,1112000.0,903999.984375,(주)LG생활건강
...,...,...,...,...,...,...,...,...
11127,70891.117188,70900.0,8.882812,(주)이마트,70891.117188,70900.0,8.882812,(주)이마트
11128,37172.367188,37150.0,22.367188,한국타이어앤테크놀로지(주),37172.367188,37150.0,22.367188,한국타이어앤테크놀로지(주)
11129,116215.335938,116200.0,15.335938,(주)오리온,116215.335938,116200.0,15.335938,(주)오리온
11130,109008.218750,109000.0,8.218750,롯데웰푸드(주),109008.218750,109000.0,8.218750,롯데웰푸드(주)


In [ ]:
#whole_data.to_csv('whole_data1.csv', encoding='utf-8-sig')

In [ ]:
whole_data.columns=['top_predict_price', 'top_real_price','top_difference' ,'top_five_name', 'bottom_predict_price', 'bottom_real_price','bottom_difference','bottom_five_name']

In [ ]:
whole_data

,top_predict_price,top_real_price,top_difference,top_five_name,bottom_predict_price,bottom_real_price,bottom_difference,bottom_five_name
0,112394.460938,96300.0,16094.460938,(주)케이티앤지,112394.460938,96300.0,16094.460938,(주)케이티앤지
1,154718.968750,16800.0,137918.968750,(주)LG유플러스,154718.968750,16800.0,137918.968750,(주)LG유플러스
2,98744.046875,91000.0,7744.046875,(주)현대백화점,98744.046875,91000.0,7744.046875,(주)현대백화점
3,151849.218750,34650.0,117199.218750,(주)한섬,151849.218750,34650.0,117199.218750,(주)한섬
4,208000.015625,1112000.0,903999.984375,(주)LG생활건강,208000.015625,1112000.0,903999.984375,(주)LG생활건강
...,...,...,...,...,...,...,...,...
11127,70891.117188,70900.0,8.882812,(주)이마트,70891.117188,70900.0,8.882812,(주)이마트
11128,37172.367188,37150.0,22.367188,한국타이어앤테크놀로지(주),37172.367188,37150.0,22.367188,한국타이어앤테크놀로지(주)
11129,116215.335938,116200.0,15.335938,(주)오리온,116215.335938,116200.0,15.335938,(주)오리온
11130,109008.218750,109000.0,8.218750,롯데웰푸드(주),109008.218750,109000.0,8.218750,롯데웰푸드(주)


# Result

In [ ]:
final_data=pd.DataFrame()
i=0
for i in range(0,1113200,44):
    df=whole_data[i:i+44][['top_predict_price','top_real_price','top_difference','top_five_name']]
    df=df.sort_values('top_difference')
    top1=df.head(1)
    bottom1=df.tail(1)
    top_bottom=pd.concat([top1, bottom1], axis=1,ignore_index=True)
    final_data=pd.concat([top_bottom, final_data], ignore_index=True)
    i=i+0

print('<top>')
print(final_data[3].value_counts())
print('*'*30)
print('<bottom>')
final_data[7].value_counts()

<top>
롯데웰푸드(주)          1104
현대글로비스(주)          993
삼성에스디에스(주)         986
(주)이마트             953
현대자동차(주)           889
(주)셀트리온            852
(주)녹십자             847
(주)아모레퍼시픽          806
(주)종근당             790
금호석유화학(주)          784
(주)오리온             758
에스케이하이닉스(주)        659
엘지이노텍(주)           658
(주)한솔케미칼           646
현대모비스(주)           616
(주)제일기획            605
(주)케이티앤지           595
삼양식품(주)            567
(주)현대백화점           565
(주)에스원             540
(주)DB하이텍           528
(주)신세계             521
(주)한섬              509
(주)현대홈쇼핑           484
(주)LG유플러스          477
(주)케이티             475
고려아연(주)            472
(주)영원무역            463
(주)녹십자홀딩스          462
피아이첨단소재(주)         450
(주)한화              441
한전KPS(주)           422
티케이지휴켐스(주)         421
롯데정밀화학(주)          416
(주)동서              409
동원시스템즈(주)          398
한국타이어앤테크놀로지(주)     397
삼성전자(주)            390
(주)지에스리테일          362
(주)오뚜기             357
(주)엔씨소프트           334
(주)농심              329
(주)LG생활건강          289
기아(주)

(주)LG생활건강         6244
(주)엔씨소프트          1051
(주)신세계             931
고려아연(주)            874
삼성에스디에스(주)         799
(주)한솔케미칼           754
(주)오뚜기             712
(주)이마트             630
(주)제일기획            618
(주)농심              613
(주)현대백화점           597
롯데웰푸드(주)           591
삼양식품(주)            567
(주)한화              563
(주)종근당             552
엘지이노텍(주)           540
(주)셀트리온            510
(주)녹십자             488
(주)케이티             459
(주)오리온             448
(주)녹십자홀딩스          426
(주)현대홈쇼핑           423
금호석유화학(주)          420
(주)아모레퍼시픽          414
한국타이어앤테크놀로지(주)     410
(주)케이티앤지           395
(주)지에스리테일          374
에스케이하이닉스(주)        359
현대자동차(주)           336
(주)동서              322
현대글로비스(주)          311
삼성전자(주)            290
(주)한섬              288
(주)LG유플러스          273
피아이첨단소재(주)         261
(주)영원무역            237
동원시스템즈(주)          226
롯데정밀화학(주)          205
(주)에스원             193
(주)DB하이텍           171
현대모비스(주)           170
한전KPS(주)           126
기아(주)               67
티케이지휴켐스(주) 

In [ ]:
# top=pd.DataFrame(whole_data['top_five_name'].value_counts())
# top.head(30)

In [ ]:
# bottom=pd.DataFrame(whole_data['bottom_five_name'].value_counts())
# bottom.head(30)

In [ ]:
#bottom.shape

In [ ]:
end=time.time()

In [ ]:
print('Time:', np.round((end-start)/3600,2),'H') ## GPU 사용됨 ##

Time: 0.2 H
